In [1]:
import os
import pyodbc
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# cnxn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};\
#                        Server=lasr-sqldb-prd-im,17001;\
#                        Database=IM_I_IDWBRRESULTS_S;\
#                        Trusted_Connection=no;\
#                        UID=KPMALC;\
#                        PWD=Summer_4;')

cnxn = pyodbc.connect('Driver={SQL Server Native Client 11.0};\
                       Server=lasr-sqldb-prd-im,17001;\
                       Database=IM_I_IDWBRRESULTS_S;\
                       Trusted_Connection=yes;')

In [6]:
sql_query_monthly = """
select
C.*
from
(
select
pfm.Aggrid,
pfm.postdt,
pfm.rtntyp,
pfm.totrtn1mo,
pfm.totrtn3mo,
pfm.totrtn6mo,
pfm.totrtn12mo,
mars.PortfolioUID,
cg.InvestmentPortfolioTypeCode,
ia.InvestmentAccountUID,
ia.InvestmentAccountTypeCode,
ia.SeparateAccountTypeCode,
ia.FundAccountTypeCode,
ia.FundAccountSubTypeCode,
ia.FundAccountGroupTypeCode
from
(
select 
aggrid,
postdt,
rtntyp,
totrtn1mo,
totrtn3mo,
totrtn6mo,
totrtn12mo
from IM_I_IDWBRRESULTS_S.dbo.MA_PP_PFMHIST
where incltyp = 'I' and crncyid = 1
) as pfm
  inner join
    (
	select
	A.AGGRID,
	A.PortfolioUID
	from
		(
		select
		AGGRID,
		PortfolioUID,
		DWUpdateDate
		from IM_I_PORTFOLIOPOS_S.dbo.MarsAggrMapping
		) as A
		inner join
		(
		select
		AGGRID,
		max(DWUpdateDate) as max_DWUpdateDate
		from IM_I_PORTFOLIOPOS_S.dbo.MarsAggrMapping
		group by AGGRID
		) as B
		on A.AGGRID=B.AGGRID and A.DWUpdateDate=B.max_DWUpdateDate
	) as mars
  on pfm.AGGRID=mars.aggrid
  left join 
     IM_I_PORTFOLIOPOS_S.dbo.cgportfolio as cg
  on mars.PortfolioUID=cg.PortfolioUID and pfm.postdt between  cg.dweffectivefromdate and cg.dweffectivetodate
  
  left join
     IM_U_RESULTS_S.KPMG.KMPYAD_AA as AA
  on cg.portfoliouid =  AA.AAPortfolioUID and pfm.postdt between  AA.dweffectivefromdate and AA.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AA'
  left join
     IM_U_RESULTS_S.KPMG.KMPYAD_AR as AR
  on cg.portfoliouid =  AR.ARPortfolioUID and pfm.postdt between  AR.dweffectivefromdate and AR.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AR'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_AT as AT
  on cg.portfoliouid =  AT.ATPortfolioUID and pfm.postdt between  AT.dweffectivefromdate and AT.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AT'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_RP as RP
  on cg.portfoliouid =  RP.RPPortfolioUID and pfm.postdt between  RP.dweffectivefromdate and RP.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'RP'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_AM as AM
  on cg.portfoliouid =  AM.AMPortfolioUID and pfm.postdt between  AM.dweffectivefromdate and AM.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AM'
  --the extra non-jasper ones
  --the extra non-jasper ones
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_GA as GA
  on cg.portfoliouid =  GA.GAPortfolioUID and pfm.postdt between  GA.dweffectivefromdate and GA.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GA'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GM as GM
  --on cg.portfoliouid =  GM.GMPortfolioUID and pfm.postdt between  GM.dweffectivefromdate and GM.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GM'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GP as GP
  --on cg.portfoliouid =  GP.GPPortfolioUID and pfm.postdt between  GP.dweffectivefromdate and GP.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GP'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_ID as ID
  on cg.portfoliouid =  ID.IDPortfolioUID and pfm.postdt between  ID.dweffectivefromdate and ID.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'ID'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_MR as MR
  --on cg.portfoliouid =  MR.MRPortfolioUID and pfm.postdt between  MR.dweffectivefromdate and MR.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'MR'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_SG as SG
  --on cg.portfoliouid =  SG.SGPortfolioUID and pfm.postdt between  SG.dweffectivefromdate and SG.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'SG'
  
   -----acct_mgr table-----
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GM as GM
  --on pfm.aggrid =  GM.GRPMGRAGGRID and pfm.postdt between  GM.STARTDT and GM.ENDDT and cg.InvestmentPortfolioTypeCode = 'GM'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GP as GP
  --on pfm.aggrid =  GP.GRPAGGRID and pfm.postdt between  GP.STARTDT and GP.ENDDT and cg.InvestmentPortfolioTypeCode = 'GP'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_GR as GR
  on pfm.aggrid =  GR.RPPCXGRPAGGRID and pfm.postdt between  GR.STARTDT and GR.ENDDT and cg.InvestmentPortfolioTypeCode = 'GR'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_MG as MG
  --on pfm.aggrid =  MG.MGRAGGRID and pfm.postdt between  MG.STARTDT and MG.ENDDT and cg.InvestmentPortfolioTypeCode = 'MG'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_RM as RM
  --on pfm.aggrid =  RM.RPPCXMGRAGGRID and pfm.postdt between  RM.STARTDT and RM.ENDDT and cg.InvestmentPortfolioTypeCode = 'RM'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_SG as SG
  --on pfm.aggrid =  SG.SPRGRPAGGRID and pfm.postdt between  SG.STARTDT and SG.ENDDT and cg.InvestmentPortfolioTypeCode = 'SG'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_SM as SM
  --on pfm.aggrid =  SM.SPRGRPMGRAGGRID and pfm.postdt between  SM.STARTDT and SM.ENDDT and cg.InvestmentPortfolioTypeCode = 'SM'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_SR as SR
  on pfm.aggrid =  SR.RPPCXSPRGRPAGGRID and pfm.postdt between  SR.STARTDT and SR.ENDDT and cg.InvestmentPortfolioTypeCode = 'SR' 
  
  
  left join
     IM_I_ACCTPOSTRANS_S.dbo.InvestmentAccount as ia
	       ---case it out---
  on ia.InvestmentAccountUID = case when cg.InvestmentPortfolioTypeCode='AA' THEN AA.InvestmentAccountUID
                                    when cg.InvestmentPortfolioTypeCode='AR' THEN AR.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='AT' THEN AT.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='RP' THEN RP.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='AM' THEN AM.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='GA' THEN GA.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='GM' THEN GM.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='GP' THEN GP.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='ID' THEN ID.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='MR' THEN MR.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='SG' THEN SG.InvestmentAccountUID
									----acct_mgr ones here----
									--when cg.InvestmentPortfolioTypeCode  = 'GM' THEN GM.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode  = 'GP' THEN GP.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode  = 'GR' THEN GR.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode  = 'MG' THEN MG.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode  = 'RM' THEN RM.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode  = 'SG' THEN SG.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode  = 'SM' THEN SM.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode  = 'SR' THEN SR.InvestmentAccountUID

								end
                                
  and pfm.postdt between ia.dweffectivefromdate and ia.dweffectivetodate
) as C
"""
# Save the data 
df_monthly = pd.read_sql(sql_query_monthly, cnxn)

In [7]:
print(df_monthly.shape)
df_monthly.head()

(17875366, 15)


,Aggrid,postdt,rtntyp,totrtn1mo,totrtn3mo,totrtn6mo,totrtn12mo,PortfolioUID,InvestmentPortfolioTypeCode,InvestmentAccountUID,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode
0,75736910,2017-09-29,M,NaN,NaN,NaN,NaN,94749,AM,10051526.0,UNAVAILABLE,None,None,None,None
1,1788140,2018-01-31,M,NaN,NaN,NaN,NaN,287943,AM,30031406.0,UNAVAILABLE,None,None,None,None
2,187790022,2017-06-30,M,100.383857,106.800016,115.527960,119.249798,290736,GA,8026508.0,UNAVAILABLE,None,None,None,None
3,15208929,2016-08-31,M,NaN,NaN,NaN,NaN,19101,AA,1068053.0,UNAVAILABLE,None,None,None,None
4,146711965,2018-08-31,M,100.878930,103.845956,103.178092,113.170716,180350,AM,7006791.0,UNAVAILABLE,None,None,None,None


In [9]:
sql_query_daily = """
select
C.*
from
(
select
pfm.Aggrid,
pfm.postdt,
pfm.rtntyp,
mars.PortfolioUID,
cg.InvestmentPortfolioTypeCode,
ia.InvestmentAccountUID,
ia.InvestmentAccountTypeCode,
ia.SeparateAccountTypeCode,
ia.FundAccountTypeCode,
ia.FundAccountSubTypeCode,
ia.FundAccountGroupTypeCode
from
(
select
aggrid,
'D' as rtntyp,
DATEADD(m, DATEDIFF(m, 0, postdt), 0) as postdt
from IM_I_IDWBRRESULTS_S.dbo.MA_PP_PFHIST
where INCLTYP = 'I' and TOTRTNDAY is not NULL  and CRNCYID = 1
group by aggrid, DATEADD(m, DATEDIFF(m, 0, postdt), 0)
having sum(1) >= 15
) as pfm
  inner join
    (
	select
	A.AGGRID,
	A.PortfolioUID
	from
		(
		select
		AGGRID,
		PortfolioUID,
		DWUpdateDate
		from IM_I_PORTFOLIOPOS_S.dbo.MarsAggrMapping
		) as A
		inner join
		(
		select
		AGGRID,
		max(DWUpdateDate) as max_DWUpdateDate
		from IM_I_PORTFOLIOPOS_S.dbo.MarsAggrMapping
		group by AGGRID
		) as B
		on A.AGGRID=B.AGGRID and A.DWUpdateDate=B.max_DWUpdateDate
	) as mars
  on pfm.AGGRID=mars.aggrid
  left join 
     IM_I_PORTFOLIOPOS_S.dbo.cgportfolio as cg
  on mars.PortfolioUID=cg.PortfolioUID and pfm.postdt between  cg.dweffectivefromdate and cg.dweffectivetodate
  
  left join
     IM_U_RESULTS_S.KPMG.KMPYAD_AA as AA
  on cg.portfoliouid =  AA.AAPortfolioUID and pfm.postdt between  AA.dweffectivefromdate and AA.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AA'
  left join
     IM_U_RESULTS_S.KPMG.KMPYAD_AR as AR
  on cg.portfoliouid =  AR.ARPortfolioUID and pfm.postdt between  AR.dweffectivefromdate and AR.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AR'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_AT as AT
  on cg.portfoliouid =  AT.ATPortfolioUID and pfm.postdt between  AT.dweffectivefromdate and AT.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AT'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_RP as RP
  on cg.portfoliouid =  RP.RPPortfolioUID and pfm.postdt between  RP.dweffectivefromdate and RP.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'RP'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_AM as AM
  on cg.portfoliouid =  AM.AMPortfolioUID and pfm.postdt between  AM.dweffectivefromdate and AM.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AM'
  --the extra non-jasper ones
  --the extra non-jasper ones
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_GA as GA
  on cg.portfoliouid =  GA.GAPortfolioUID and pfm.postdt between  GA.dweffectivefromdate and GA.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GA'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GM as GM
  --on cg.portfoliouid =  GM.GMPortfolioUID and pfm.postdt between  GM.dweffectivefromdate and GM.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GM'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GP as GP
  --on cg.portfoliouid =  GP.GPPortfolioUID and pfm.postdt between  GP.dweffectivefromdate and GP.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GP'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_ID as ID
  on cg.portfoliouid =  ID.IDPortfolioUID and pfm.postdt between  ID.dweffectivefromdate and ID.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'ID'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_MR as MR
  --on cg.portfoliouid =  MR.MRPortfolioUID and pfm.postdt between  MR.dweffectivefromdate and MR.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'MR'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_SG as SG
  --on cg.portfoliouid =  SG.SGPortfolioUID and pfm.postdt between  SG.dweffectivefromdate and SG.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'SG'

-----acct_mgr table-----
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GM as GM
  --on pfm.aggrid =  GM.GRPMGRAGGRID and pfm.postdt between  GM.STARTDT and GM.ENDDT and cg.InvestmentPortfolioTypeCode = 'GM'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GP as GP
  --on pfm.aggrid =  GP.GRPAGGRID and pfm.postdt between  GP.STARTDT and GP.ENDDT and cg.InvestmentPortfolioTypeCode = 'GP'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_GR as GR
  on pfm.aggrid =  GR.RPPCXGRPAGGRID and pfm.postdt between  GR.STARTDT and GR.ENDDT and cg.InvestmentPortfolioTypeCode = 'GR'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_MG as MG
  --on pfm.aggrid =  MG.MGRAGGRID and pfm.postdt between  MG.STARTDT and MG.ENDDT and cg.InvestmentPortfolioTypeCode = 'MG'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_RM as RM
  --on pfm.aggrid =  RM.RPPCXMGRAGGRID and pfm.postdt between  RM.STARTDT and RM.ENDDT and cg.InvestmentPortfolioTypeCode = 'RM'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_SG as SG
  --on pfm.aggrid =  SG.SPRGRPAGGRID and pfm.postdt between  SG.STARTDT and SG.ENDDT and cg.InvestmentPortfolioTypeCode = 'SG'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_SM as SM
  --on pfm.aggrid =  SM.SPRGRPMGRAGGRID and pfm.postdt between  SM.STARTDT and SM.ENDDT and cg.InvestmentPortfolioTypeCode = 'SM'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_SR as SR
  on pfm.aggrid =  SR.RPPCXSPRGRPAGGRID and pfm.postdt between  SR.STARTDT and SR.ENDDT and cg.InvestmentPortfolioTypeCode = 'SR'

  left join
     IM_I_ACCTPOSTRANS_S.dbo.InvestmentAccount as ia
	       ---case it out---
  on ia.InvestmentAccountUID = case when cg.InvestmentPortfolioTypeCode='AA' THEN AA.InvestmentAccountUID
                                    when cg.InvestmentPortfolioTypeCode='AR' THEN AR.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='AT' THEN AT.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='RP' THEN RP.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='AM' THEN AM.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='GA' THEN GA.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='GM' THEN GM.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='GP' THEN GP.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='ID' THEN ID.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='MR' THEN MR.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='SG' THEN SG.InvestmentAccountUID
									----acct_mgr ones here----
									--when cg.InvestmentPortfolioTypeCode  = 'GM' THEN GM.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode  = 'GP' THEN GP.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode  = 'GR' THEN GR.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode  = 'MG' THEN MG.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode  = 'RM' THEN RM.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode  = 'SG' THEN SG.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode  = 'SM' THEN SM.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode  = 'SR' THEN SR.InvestmentAccountUID

								end
  and pfm.postdt between ia.dweffectivefromdate and ia.dweffectivetodate
) as C
"""
# Save the data 
df_daily = pd.read_sql(sql_query_daily, cnxn)

In [11]:
cnxn.close()

In [12]:
print(df_monthly.shape)
df_monthly.head()

(17875366, 15)


,Aggrid,postdt,rtntyp,totrtn1mo,totrtn3mo,totrtn6mo,totrtn12mo,PortfolioUID,InvestmentPortfolioTypeCode,InvestmentAccountUID,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode
0,75736910,2017-09-29,M,NaN,NaN,NaN,NaN,94749,AM,10051526.0,UNAVAILABLE,None,None,None,None
1,1788140,2018-01-31,M,NaN,NaN,NaN,NaN,287943,AM,30031406.0,UNAVAILABLE,None,None,None,None
2,187790022,2017-06-30,M,100.383857,106.800016,115.527960,119.249798,290736,GA,8026508.0,UNAVAILABLE,None,None,None,None
3,15208929,2016-08-31,M,NaN,NaN,NaN,NaN,19101,AA,1068053.0,UNAVAILABLE,None,None,None,None
4,146711965,2018-08-31,M,100.878930,103.845956,103.178092,113.170716,180350,AM,7006791.0,UNAVAILABLE,None,None,None,None


In [13]:
print(df_daily.shape)
df_daily.head()

(9396923, 11)


,Aggrid,postdt,rtntyp,PortfolioUID,InvestmentPortfolioTypeCode,InvestmentAccountUID,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode
0,91215789,2003-05-01,D,114136,AM,198100898.0,AG,None,None,None,None
1,89215787,2004-11-01,D,111609,AM,198100898.0,AG,None,None,None,None
2,132327884,2014-07-01,D,163680,AM,115100601.0,AG,None,None,None,None
3,149767936,2015-12-01,D,184164,AT,27066171.0,AG,None,None,None,None
4,208320116,2013-04-01,D,250182,AA,115100601.0,AG,None,None,None,None


In [14]:
def align_values(row):
    if row['rtntyp'] == 'M':
        return row['totrtn1mo']
    elif row['rtntyp'] == 'Q':
        return row['totrtn3mo']
    elif row['rtntyp'] == 'S':
        return row['totrtn6mo']
    elif row['rtntyp'] == 'A':
        return row['totrtn12mo']
    else:
        return np.nan

In [16]:
df_monthly['rtntyp_values'] = df_monthly.apply(lambda x: align_values(x), axis=1)

In [17]:
df_monthly.shape

(17875366, 16)

In [18]:
df_monthly = df_monthly[['Aggrid', 'postdt', 'rtntyp', 'rtntyp_values', 
                         'InvestmentPortfolioTypeCode', 'InvestmentAccountTypeCode', 
                         'SeparateAccountTypeCode', 'FundAccountTypeCode',
                         'FundAccountSubTypeCode', 'FundAccountGroupTypeCode']]

In [19]:
# Drop rows with NULL values
df_monthly = df_monthly[df_monthly['rtntyp_values'].notnull()]

print(df_monthly.shape)
df_monthly.head()

(11312762, 10)


,Aggrid,postdt,rtntyp,rtntyp_values,InvestmentPortfolioTypeCode,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode
2,187790022,2017-06-30,M,100.383857,GA,UNAVAILABLE,None,None,None,None
4,146711965,2018-08-31,M,100.878930,AM,UNAVAILABLE,None,None,None,None
5,198318622,2017-10-31,M,106.558861,AM,FUND,None,MF,AF,NG
6,191731302,2010-02-26,M,100.715903,AT,AG,None,None,None,None
7,161724488,2012-05-31,M,95.707510,AA,AG,None,None,None,None


In [20]:
df_monthly.drop(['rtntyp_values'], axis=1, inplace=True)
df_monthly.head()

,Aggrid,postdt,rtntyp,InvestmentPortfolioTypeCode,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode
2,187790022,2017-06-30,M,GA,UNAVAILABLE,None,None,None,None
4,146711965,2018-08-31,M,AM,UNAVAILABLE,None,None,None,None
5,198318622,2017-10-31,M,AM,FUND,None,MF,AF,NG
6,191731302,2010-02-26,M,AT,AG,None,None,None,None
7,161724488,2012-05-31,M,AA,AG,None,None,None,None


In [21]:
new_aggrid_list = []
new_date_list = []
new_rtntyp_list = []
new_hier_list = []
new_invport_list = [] #InvestmentPortfolioTypeCode,
new_lab1 = [] #InvestmentAccountTypeCode,
new_lab2 = [] #SeparateAccountTypeCode,
new_lab3 = [] #FundAccountTypeCode,
new_lab4 = [] #FundAccountSubTypeCode,
new_lab5 = [] #FundAccountGroupTypeCode

for aggrid, postdt, rtntyp, InvestmentPortfolioTypeCode, InvestmentAccountTypeCode, SeparateAccountTypeCode, FundAccountTypeCode, FundAccountSubTypeCode, FundAccountGroupTypeCode in df_monthly.itertuples(index=False):
    if rtntyp == 'A':
        for d in pd.date_range(end=postdt, periods=12, freq='MS'):
            new_aggrid_list.append(aggrid)
            new_date_list.append(d)
            new_rtntyp_list.append(rtntyp)
            new_hier_list.append(4)
            
            new_invport_list.append(InvestmentPortfolioTypeCode)
            new_lab1.append(InvestmentAccountTypeCode)
            new_lab2.append(SeparateAccountTypeCode)
            new_lab3.append(FundAccountTypeCode)
            new_lab4.append(FundAccountSubTypeCode)
            new_lab5.append(FundAccountGroupTypeCode)
    if rtntyp == 'S':
        for d in pd.date_range(end=postdt, periods=6, freq='MS'):
            new_aggrid_list.append(aggrid)
            new_date_list.append(d)
            new_rtntyp_list.append(rtntyp)
            new_hier_list.append(3)
            
            new_invport_list.append(InvestmentPortfolioTypeCode)
            new_lab1.append(InvestmentAccountTypeCode)
            new_lab2.append(SeparateAccountTypeCode)
            new_lab3.append(FundAccountTypeCode)
            new_lab4.append(FundAccountSubTypeCode)
            new_lab5.append(FundAccountGroupTypeCode)
    if rtntyp == 'Q':
        for d in pd.date_range(end=postdt, periods=3, freq='MS'):
            new_aggrid_list.append(aggrid)
            new_date_list.append(d)
            new_rtntyp_list.append(rtntyp)
            new_hier_list.append(2)
            
            new_invport_list.append(InvestmentPortfolioTypeCode)
            new_lab1.append(InvestmentAccountTypeCode)
            new_lab2.append(SeparateAccountTypeCode)
            new_lab3.append(FundAccountTypeCode)
            new_lab4.append(FundAccountSubTypeCode)
            new_lab5.append(FundAccountGroupTypeCode)
    if rtntyp == 'M':
            new_aggrid_list.append(aggrid)
            new_date_list.append(postdt.replace(day=1))
            new_rtntyp_list.append(rtntyp)
            new_hier_list.append(1)
            
            new_invport_list.append(InvestmentPortfolioTypeCode)
            new_lab1.append(InvestmentAccountTypeCode)
            new_lab2.append(SeparateAccountTypeCode)
            new_lab3.append(FundAccountTypeCode)
            new_lab4.append(FundAccountSubTypeCode)
            new_lab5.append(FundAccountGroupTypeCode)

df_monthly_1stm = pd.DataFrame(columns=['aggrid', 'postdt', 'rtntyp'])
df_monthly_1stm['aggrid'] = new_aggrid_list
df_monthly_1stm['postdt'] = new_date_list
df_monthly_1stm['rtntyp'] = new_rtntyp_list
df_monthly_1stm['hier'] = new_hier_list

#new fields
df_monthly_1stm['PortType'] = new_invport_list
df_monthly_1stm['lab1'] = new_lab1
df_monthly_1stm['lab2'] = new_lab2
df_monthly_1stm['lab3'] = new_lab3
df_monthly_1stm['lab4'] = new_lab4
df_monthly_1stm['lab5'] = new_lab5

In [22]:
print(df_monthly_1stm.shape)
df_monthly_1stm.head()

(11428564, 10)


,aggrid,postdt,rtntyp,hier,PortType,lab1,lab2,lab3,lab4,lab5
0,187790022,2017-06-01,M,1,GA,UNAVAILABLE,None,None,None,None
1,146711965,2018-08-01,M,1,AM,UNAVAILABLE,None,None,None,None
2,198318622,2017-10-01,M,1,AM,FUND,None,MF,AF,NG
3,191731302,2010-02-01,M,1,AT,AG,None,None,None,None
4,161724488,2012-05-01,M,1,AA,AG,None,None,None,None


In [23]:
#check no duplicate records
print(df_monthly_1stm.drop_duplicates(subset=['aggrid', 'postdt'], keep='first', inplace=False).count())

aggrid      11098066
postdt      11098066
rtntyp      11098066
hier        11098066
PortType    11098066
lab1         6520970
lab2          109511
lab3          130450
lab4           79437
lab5           95946
dtype: int64


In [24]:
#there are a small few - remove them
df_monthly_1stm.drop_duplicates(subset=['aggrid', 'postdt'], keep='first', inplace=True)
print(df_monthly_1stm.shape)
df_monthly_1stm.head()

(11098066, 10)


,aggrid,postdt,rtntyp,hier,PortType,lab1,lab2,lab3,lab4,lab5
0,187790022,2017-06-01,M,1,GA,UNAVAILABLE,None,None,None,None
1,146711965,2018-08-01,M,1,AM,UNAVAILABLE,None,None,None,None
2,198318622,2017-10-01,M,1,AM,FUND,None,MF,AF,NG
3,191731302,2010-02-01,M,1,AT,AG,None,None,None,None
4,161724488,2012-05-01,M,1,AA,AG,None,None,None,None


In [25]:
df_monthly_1stm = df_monthly_1stm.rename(columns={'PortType':'Portype'})

In [26]:
#df_daily.rename(columns={'postdt_m': 'postdt'}, inplace=True)
df_daily = df_daily.rename(columns={
    'InvestmentPortfolioTypeCode':'Portype',
    'InvestmentAccountTypeCode':'lab1',
    'SeparateAccountTypeCode':'lab2',
    'FundAccountTypeCode':'lab3',
    'FundAccountSubTypeCode':'lab4',
    'FundAccountGroupTypeCode':'lab5',
})

df_daily['hier']=0
df_daily.head()

,Aggrid,postdt,rtntyp,PortfolioUID,Portype,InvestmentAccountUID,lab1,lab2,lab3,lab4,lab5,hier
0,91215789,2003-05-01,D,114136,AM,198100898.0,AG,None,None,None,None,0
1,89215787,2004-11-01,D,111609,AM,198100898.0,AG,None,None,None,None,0
2,132327884,2014-07-01,D,163680,AM,115100601.0,AG,None,None,None,None,0
3,149767936,2015-12-01,D,184164,AT,27066171.0,AG,None,None,None,None,0
4,208320116,2013-04-01,D,250182,AA,115100601.0,AG,None,None,None,None,0


In [27]:
df_daily = df_daily.drop(['InvestmentAccountUID', 'PortfolioUID'], axis=1)

In [28]:
df_daily = df_daily.rename(columns={'Aggrid':'aggrid'})

In [29]:
df_daily.head()

,aggrid,postdt,rtntyp,Portype,lab1,lab2,lab3,lab4,lab5,hier
0,91215789,2003-05-01,D,AM,AG,None,None,None,None,0
1,89215787,2004-11-01,D,AM,AG,None,None,None,None,0
2,132327884,2014-07-01,D,AM,AG,None,None,None,None,0
3,149767936,2015-12-01,D,AT,AG,None,None,None,None,0
4,208320116,2013-04-01,D,AA,AG,None,None,None,None,0


# COMBINE THEM

In [30]:
df_combo = df_monthly_1stm.append(df_daily, ignore_index=True)
print(df_combo.shape)
df_combo.head()

(20494989, 10)


,Portype,aggrid,hier,lab1,lab2,lab3,lab4,lab5,postdt,rtntyp
0,GA,187790022,1,UNAVAILABLE,None,None,None,None,2017-06-01,M
1,AM,146711965,1,UNAVAILABLE,None,None,None,None,2018-08-01,M
2,AM,198318622,1,FUND,None,MF,AF,NG,2017-10-01,M
3,AT,191731302,1,AG,None,None,None,None,2010-02-01,M
4,AA,161724488,1,AG,None,None,None,None,2012-05-01,M


In [31]:
df_left = df_combo.groupby(['aggrid', 'postdt'])['hier'].min().reset_index()
print(df_left.shape)

(11180558, 3)


In [32]:
df_final = df_left.merge(df_combo, left_on=['aggrid', 'postdt', 'hier'], right_on=['aggrid', 'postdt', 'hier'], how='left')

#df_final.drop(['hier'], axis=1, inplace=True)
print(df_final.shape)
df_final.head()

(11452930, 10)


,aggrid,postdt,hier,Portype,lab1,lab2,lab3,lab4,lab5,rtntyp
0,10600,1975-01-01,4,AA,MF,None,None,None,None,A
1,10600,1975-02-01,4,AA,MF,None,None,None,None,A
2,10600,1975-03-01,4,AA,MF,None,None,None,None,A
3,10600,1975-04-01,4,AA,MF,None,None,None,None,A
4,10600,1975-05-01,4,AA,MF,None,None,None,None,A


In [ ]:
df_final.to_csv('PF_Daily_n_Monthly_w_Labels_20181008.csv')

# Summarized Version

In [44]:
df_final_filled = df_final.fillna('NA')
df_summary_out = df_final_filled.groupby(['postdt','rtntyp', 'Portype', 'lab1', 'lab2', 'lab3', 'lab4', 'lab5'])['aggrid'].count().reset_index()
df_summary_out.rename(columns={'aggrid': 'aggrid_cnt'}, inplace=True)
print(df_summary_out.shape)
df_summary_out.head()

(37352, 9)


,postdt,rtntyp,Portype,lab1,lab2,lab3,lab4,lab5,aggrid_cnt
0,1934-01-01,M,CP,NA,NA,NA,NA,NA,2
1,1934-01-01,M,FL,NA,NA,NA,NA,NA,2
2,1934-02-01,M,CP,NA,NA,NA,NA,NA,2
3,1934-02-01,M,FL,NA,NA,NA,NA,NA,2
4,1934-03-01,M,CP,NA,NA,NA,NA,NA,2


In [45]:
df_summary_out.to_csv('PF_Daily_n_Monthly_w_Labels_Summarized_20181008.csv')

In [47]:
df_summary_out.aggrid_cnt.sum()

11452930